In [5]:
'''
# 建立SQL連線權限

CREATE USER 'yy_jimmy'@'127.0.0.1' IDENTIFIED BY 'yyjimmy';
GRANT ALL PRIVILEGES ON yyostech.* TO 'yy_jimmy'@'127.0.0.1';
FLUSH PRIVILEGES;
'''

# 測試連線
import pymysql
# 連線資料庫
db = pymysql.connect(host='127.0.0.1',
                     port=3306,
                     user='yy_jimmy',
                     password='yyjimmy',
                     db='yyostech')
# 建立連線物件
cursor = db.cursor()
# 新增資料表
# user_right
cursor.execute("""select `line_id` from user_profile where `line_id` = '1234'""")
# 使用 fetchone() 方法获取单条数据.
data = cursor.fetchone()
 
print(data)
 
# 关闭数据库连接
db.close()

None


In [3]:
# 寫入資料
import pymysql
 
# 連線資料庫
db = pymysql.connect(host='127.0.0.1',
                     port=3306,
                     user='yy_jimmy',
                     password='yyjimmy',
                     db='yyostech')
# 建立連線物件
cursor = db.cursor()
sql = """INSERT INTO user_right (right_id, menu_id, menu_image) VALUES(%s, %s, %s);"""

right_id = 0
menu_id = 'richmenu-6273c8438983aceac1d6ec86e304c169'
with open('./lineMenu1.png', 'rb') as file:
    menu_image = file.read()
    
data = (right_id, menu_id, menu_image)
cursor.execute(sql, data)
# 確認執行上述指令
result = db.commit()
# 回傳結果(none為成功)
print (result)
 
# 關閉資料庫連線
db.close()

None


D:\python\factory\venv\lib\site-packages\pymysql\cursors.py:170: Warning: (1300, "Invalid utf8mb4 character string: '89504E'")
  result = self._query(query)


In [4]:
'''
取得資料庫圖檔
'''
import pymysql
import cv2
import numpy as np

# 打开数据库连接
db = pymysql.connect(host='127.0.0.1',
                     port=3306,
                     user='yy_jimmy',
                     password='yyjimmy',
                     db='yyostech')
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 execute()  方法执行 SQL 查询 
cursor.execute("SELECT a.image, a.plant_percent from sponge_record a inner join artificial_identify b on a.sponge_id = b.sponge_id where artificial_id = 29")
# 使用 fetchone() 方法获取单条数据.
data = cursor.fetchone()
# 关闭数据库连接
db.close()
print(data[1])
frame = data[0]
# converting bytestring frame into imshow argument
nparr = np.fromstring(frame, np.uint8)
frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
cv2.namedWindow("rect",0)
cv2.resizeWindow('rect', 800,500)
cv2.imshow('rect', frame)
cv2.waitKey(0)

0.0


D:\python\factory\venv\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


27

In [2]:
# mysql建立資料表

import pymysql
 
# 連線資料庫
db = pymysql.connect(host='127.0.0.1',
                     port=3306,
                     user='yy_jimmy',
                     password='yyjimmy',
                     db='yyostech')
# 建立連線物件
cursor = db.cursor()
 
# 新增資料表
# user_right
cursor.execute("""CREATE TABLE If not exists user_right (`right_id` INT PRIMARY KEY, 
                                           `menu_id` VARCHAR(255), 
                                           `menu_image` LONGBLOB) CHARACTER SET=utf8""")
# series_data
cursor.execute("""CREATE TABLE If not exists series_data (`series_id` VARCHAR(255) PRIMARY KEY, 
                                            `series_name` VARCHAR(255), 
                                            `image_url` VARCHAR(255), 
                                            `threshold` INT, 
                                            `germination_percent` FLOAT, 
                                            `saw_days` INT) CHARACTER SET=utf8""")
# raspi_data
cursor.execute("""CREATE TABLE If not exists raspi_data (`photo_id` VARCHAR(255) PRIMARY KEY, 
                                           `image` LONGBLOB, 
                                           `photo_date` DATETIME) CHARACTER SET=utf8""")
# user_profile
cursor.execute("""CREATE TABLE If not exists user_profile (`line_id` VARCHAR(255) PRIMARY KEY, 
                                             `user_name` VARCHAR(255), 
                                             `right_id` INT,
                  FOREIGN KEY (`right_id`) REFERENCES user_right(right_id)) CHARACTER SET=utf8;""")
# process_record
cursor.execute("""CREATE TABLE If not exists process_record (`process_id` VARCHAR(255) PRIMARY KEY, 
                                             `series_id` VARCHAR(255), 
                                             `seed_numbers` VARCHAR(255),
                                             `identify_date` DATETIME,
                                             `saw_date` DATETIME,                  
                 FOREIGN KEY (`series_id`) REFERENCES series_data(series_id)) CHARACTER SET=utf8;""")
# germiantion_record
cursor.execute("""CREATE TABLE If not exists germination_record (`germination_id` VARCHAR(255) PRIMARY KEY, 
                                                  `process_id` VARCHAR(255),
                                                  `photo_id` VARCHAR(255), 
                                                  `image` LONGBLOB, 
                                                  `germination_rate` FLOAT,
                                                  `sponge_cnt` INT,
                  FOREIGN KEY (`photo_id`) REFERENCES raspi_data(photo_id),
                  FOREIGN KEY (`process_id`) REFERENCES process_record(process_id)) CHARACTER SET=utf8""")
# sponge_record
cursor.execute("""CREATE TABLE If not exists sponge_record (`sponge_id` INT PRIMARY KEY AUTO_INCREMENT, 
                                                   `germination_id` VARCHAR(255), 
                                                   `image` LONGBLOB, 
                                                   `x_position` int,
                                                   `y_position` int,
                                                   `plant_percent` FLOAT,
                                                   `is_germination` BOOLEAN,
                  FOREIGN KEY (`germination_id`) REFERENCES germination_record(germination_id)) CHARACTER SET=utf8""")
# artificial_identify
cursor.execute("""CREATE TABLE If not exists artificial_identify (`artificial_id` INT PRIMARY KEY AUTO_INCREMENT, 
                                                    `sponge_id` INT,
                                                    `line_id` VARCHAR(255),
                                                    `is_germination` BOOLEAN, 
                                                    `identify_date` DATETIME, 
                  FOREIGN KEY (`sponge_id`) REFERENCES sponge_record(sponge_id),
                  FOREIGN KEY (`line_id`) REFERENCES user_profile(line_id)) CHARACTER SET=utf8""")
# 確認執行上述指令
result = db.commit()
# 回傳結果(none為成功)
print (result)
 
# 關閉資料庫連線
db.close()


None


In [1]:
# 呼叫樹莓派執行拍照並回傳資料的動作
import paramiko

def par(photo_ids):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("192.168.0.30",22,"pi", "Jimmy8193026")  # 網路連線設定
    stdin, stdout, stderr = ssh.exec_command("python3 /home/pi/Desktop/test.py %s" % ids)  # 呼叫執行檔並傳送參數
    paramiko_result = stdout.readlines()[0]  # 回傳產出結果
    ssh.close()
    return paramiko_result

In [14]:
# 品種參數
dict1 = [['1G1', '綠火焰',180, 1.4], 
         ['1G4', '奶油波士頓', 185, 2.0], 
         ['1G7', '蘿蔓', 180, 2.1], 
         ['1R7', '紅蘿蔓', 180, 3.0], 
         ['1RB', '紅寶石', 170, 3.0], 
         ['2GD', '羽衣甘藍', 180, 4.0], 
         ['3G1', '芝麻葉', 180, 1.5], 
         ['3G2', '蘿勒', 180, 0.8]]
dict1

[['1G1', '綠火焰', 180, 1.4],
 ['1G4', '奶油波士頓', 185, 2.0],
 ['1G7', '蘿蔓', 180, 2.1],
 ['1R7', '紅蘿蔓', 180, 3.0],
 ['1RB', '紅寶石', 170, 3.0],
 ['2GD', '羽衣甘藍', 180, 4.0],
 ['3G1', '芝麻葉', 180, 1.5],
 ['3G2', '蘿勒', 180, 0.8]]

In [15]:
# 寫入資料
import pymysql
 
# 連線資料庫
db = pymysql.connect(host='127.0.0.1',
                     port=3306,
                     user='yy_jimmy',
                     password='yyjimmy',
                     db='yyostech')
# 建立連線物件
cursor = db.cursor()
sql = """INSERT INTO series_data (series_id, series_name, threshold, germination_percent, saw_days) 
                                    VALUES(%s, %s, %s, %s, %s);"""
for i in dict1:
    data = (i[0], i[1], i[2], i[3], 6)
    cursor.execute(sql, data)
# 確認執行上述指令
result = db.commit()
# 回傳結果(none為成功)
print (result)
 
# 關閉資料庫連線
db.close()

None


In [179]:
import pymysql
import cv2
import numpy as np


class connect():
    def __init__(self, ip, port, username, password, database):
        self.ip = ip
        self.port = port
        self.username = username
        self.password = password
        self.database = database
        
    def connect_DB(self):
        db = pymysql.connect(host=self.ip,
                             port=self.port,
                             user=self.username,
                             password=self.password,
                             db=self.database)
        cursor = db.cursor()
        return db, cursor
    
    
    def convert_image_to_byte(self, image):
        is_success, im_buf_arr = cv2.imencode(".jpg", image)
        byte_im = im_buf_arr.tobytes()
        return byte_im
    
    
    def get_series(self):
        try:
            db, cursor = self.connect_DB()
            cursor.execute('''select * from series_data''')
            series_data = cursor.fetchall()
        finally:
            db.close()
        return series_data
        
        
    def save_process(self, gerData):
        try:
            db, cursor = self.connect_DB()
            sql = """INSERT INTO process_record (process_id, series_id, seed_numbers, identify_date, saw_date) 
                                        VALUES(%s, %s, %s, %s, %s);"""
            data = (gerData['process_id'], gerData['series_id'], gerData['seed_numbers'], gerData['identify_date'], gerData['saw_date'])
            cursor.execute(sql, data)
            result = db.commit()
        finally:
            db.close()
        return result
    
        
    def get_raspi_data(self, photo_id):
        try:
            db, cursor = self.connect_DB()
            cursor.execute("SELECT * from raspi_data where photo_id = \'%s\'" % photo_id)
            data = cursor.fetchone()
            frame = data[1]
            nparr = np.fromstring(frame, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            previewSize = (int(round(frame.shape[1]/15, 0)), int(round(frame.shape[0]/15, 0)))
            previewImage = cv2.resize(frame, previewSize)
            cv2.imwrite(".\YY\preview.jpg", previewImage)
        finally:
            db.close()
        return frame
    
    
    def save_germination_record(self, germination_record, result_list):
        try:
            db, cursor = self.connect_DB()
            germination_record[3] = self.convert_image_to_byte(germination_record[3])
            sql_germination = """INSERT INTO germination_record (germination_id, process_id, photo_id, image, germination_rate, sponge_cnt) 
                                        VALUES(%s, %s, %s, %s, %s, %s);"""
            cursor.execute(sql_germination, tuple(germination_record))
            sql_sponge = """INSERT INTO sponge_record (germination_id, x_position, y_position, plant_percent, is_germination, image) 
                                        VALUES(%s, %s, %s, %s, %s, %s);"""
            germination_id = germination_record[0]
            for i in result_list:
                i[4] = self.convert_image_to_byte(i[4])
                cursor.execute(sql_sponge, (germination_id, ) + tuple(i))
            result = db.commit()
        finally:
            db.close()
        return result
    
    
    def get_result(self, process_id):
        try:
            db, cursor = self.connect_DB()
            cursor.execute('''select a.process_id, 
                                count(distinct b.germination_id) as germination_cnt, 
                                count(distinct c.sponge_id) as sponge_cnt, 
                                count(if(is_germination = 1, 1, null)) as is_ger, 
                                count(if(is_germination = 0, 1, null)) as no_ger,
                                round((count(if(is_germination = 0, 1, null)) / count(distinct c.sponge_id))*100, 2) as percent
                                from process_record a
                                inner join germination_record b on a.process_id = b.process_id
                                inner join sponge_record c on c.germination_id = b.germination_id
                                where a.process_id = %s
                                group by a.process_id
                                ''' % repr(process_id))
            process_result = cursor.fetchone()
        finally:
            db.close()
        return process_result
    
    

In [173]:
# 取得品種資料
a = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')
series_data = a.get_series()
series_data

(('1G1', '綠火焰', 180, 1.4, 6),
 ('1G4', '奶油波士頓', 185, 2.0, 6),
 ('1G7', '蘿蔓', 180, 2.1, 6),
 ('1R7', '紅蘿蔓', 180, 3.0, 6),
 ('1RB', '紅寶石', 170, 3.0, 6),
 ('2GD', '羽衣甘藍', 180, 4.0, 6),
 ('3G1', '芝麻葉', 180, 1.5, 6),
 ('3G2', '蘿勒', 180, 0.8, 6))

In [174]:
# 儲存操作流程
from datetime import datetime
gerData = {'process_id': 'test1235', 
           'series_id': '1G1', 
           'seed_numbers': '1234567890', 
           'identify_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"), 
           'saw_date': '2020-12-20'}
gerData
a = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')
a.save_process(gerData)

In [180]:
# 取得pi的照片
photo_id = 'testphoto12345'
a = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')

image = a.get_raspi_data(photo_id)  

previewSize = (int(round(image.shape[1]/15, 0)), int(round(image.shape[0]/15, 0)))
cv2.namedWindow("rect",0)
cv2.resizeWindow('rect', previewSize[0],previewSize[1])
cv2.imshow('rect', image)
cv2.waitKey(0)

D:\python\factory\venv\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


-1

In [181]:
# 儲存結果
result_list = [[1, 2, 0.0, 0, image], [2, 3, 2.9, 1, image], [5, 6, 3.8, 1, image]]
ger_record = ['gertest05', 'test1235', 'testphoto12345', image, 98.7, 400]
a = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')
a.save_germination_record(ger_record, result_list)

In [182]:
# 取出本流程統整結果
a = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')
process_id = 'test1235'
result = a.get_result(process_id)
result

('test1235', 1, 3, 2, 1, Decimal('33.33'))

In [7]:
from yyGermination import connect, par, germination
connect = connect('127.0.0.1', 3306, 'yy_jimmy', 'yyjimmy', 'yyostech')
par = par("192.168.0.30",22,"pi", "Jimmy8193026")
germination = germination('3G1', 170 ,3.0)
print(germination)

作物編號: 3G1
閾值參數: 170
判斷條件: 3.0
結果暫存清單: 共0筆數據(詳細請看self.result_list)


In [6]:
import cv2
photo_id = 'test0326'
par.connect_raspi(photo_id)
image = connect.get_raspi_data(photo_id)
previewSize = (int(round(image.shape[1]/15, 0)), int(round(image.shape[0]/15, 0)))
cv2.namedWindow("rect",0)
cv2.resizeWindow('rect', previewSize[0],previewSize[1])
cv2.imshow('rect', image)
cv2.waitKey(0)

['OK\n']


-1

In [8]:
series = connect.get_series()
series

(('1G1', '綠火焰', 180, 1.4, 6),
 ('1G4', '奶油波士頓', 185, 2.0, 6),
 ('1G7', '蘿蔓', 180, 2.1, 6),
 ('1R7', '紅蘿蔓', 180, 3.0, 6),
 ('1RB', '紅寶石', 170, 3.0, 6),
 ('2GD', '羽衣甘藍', 180, 4.0, 6),
 ('3G1', '芝麻葉', 180, 1.5, 6),
 ('3G2', '蘿勒', 180, 0.8, 6))